In [1]:
import pandas as pd
import yfinance as yf

# Download stock price data for AAPL (or any stock)
stock_data = yf.download('AAPL', start='2020-01-01', end='2025-01-01')

# Reset index and rename date column
stock_data.reset_index(inplace=True)
stock_data.rename(columns={'Date': 'date'}, inplace=True)

# Convert to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

stock_data



[*********************100%***********************]  1 of 1 completed


,date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,72.796036,135480400
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.088295,146322800
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,72.662720,118387200
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.320961,108872000
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,73.484352,132079200
...,...,...,...,...,...,...,...
1253,2024-12-24,255.490005,258.209991,255.289993,258.200012,258.200012,23234700
1254,2024-12-26,258.190002,260.100006,257.630005,259.019989,259.019989,27237100
1255,2024-12-27,257.829987,258.700012,253.059998,255.589996,255.589996,42355300
1256,2024-12-30,252.229996,253.500000,250.750000,252.199997,252.199997,35557500


In [2]:
# Load the news sentiment dataset
news_data = pd.read_csv('all-data.csv', encoding='ISO-8859-1', header=None)

# Assign proper column names
news_data.columns = ['sentiment', 'news_headline']

news_data

,sentiment,news_headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [3]:
# Get stock market dates
market_dates = stock_data['date'].sort_values().reset_index(drop=True)

# Trim news data to match stock market dates length
news_data = news_data.iloc[:len(market_dates)].copy()

# Assign stock market dates to news data
news_data['date'] = market_dates

news_data


,sentiment,news_headline,date
0,neutral,"According to Gran , the company has no plans t...",2020-01-02
1,neutral,Technopolis plans to develop in stages an area...,2020-01-03
2,negative,The international electronic industry company ...,2020-01-06
3,positive,With the new production plant the company woul...,2020-01-07
4,positive,According to the company 's updated strategy f...,2020-01-08
...,...,...,...
1253,positive,"In the Homeware area , Fiskars provides leadin...",2024-12-24
1254,neutral,"In the survey , Goodyear is the third most val...",2024-12-26
1255,neutral,"In today s business , you have to pre-empt wha...",2024-12-27
1256,neutral,Ingen is an established medical device manufac...,2024-12-30


In [4]:
# Ensure stock market dates are sorted
market_dates = stock_data['date'].sort_values().reset_index(drop=True)

# Trim news data to match stock market dates length
news_data = news_data.iloc[:len(market_dates)].copy()

# Assign stock market dates to news data
news_data['date'] = market_dates

news_data


,sentiment,news_headline,date
0,neutral,"According to Gran , the company has no plans t...",2020-01-02
1,neutral,Technopolis plans to develop in stages an area...,2020-01-03
2,negative,The international electronic industry company ...,2020-01-06
3,positive,With the new production plant the company woul...,2020-01-07
4,positive,According to the company 's updated strategy f...,2020-01-08
...,...,...,...
1253,positive,"In the Homeware area , Fiskars provides leadin...",2024-12-24
1254,neutral,"In the survey , Goodyear is the third most val...",2024-12-26
1255,neutral,"In today s business , you have to pre-empt wha...",2024-12-27
1256,neutral,Ingen is an established medical device manufac...,2024-12-30


In [5]:
import ta

# Ensure no missing values in 'Close'
stock_data['Close'].fillna(method='ffill', inplace=True)

# Calculate Simple Moving Average (SMA)
stock_data['SMA_20'] = ta.trend.sma_indicator(stock_data['Close'], window=20)

# Calculate Exponential Moving Average (EMA)
stock_data['EMA_50'] = ta.trend.ema_indicator(stock_data['Close'], window=50)

# Calculate Relative Strength Index (RSI)
stock_data['RSI'] = ta.momentum.rsi(stock_data['Close'], window=14)

# Calculate Bollinger Bands
bollinger = ta.volatility.BollingerBands(stock_data['Close'], window=20)
stock_data['Bollinger_High'] = bollinger.bollinger_hband()
stock_data['Bollinger_Low'] = bollinger.bollinger_lband()

# Calculate MACD
stock_data['MACD'] = ta.trend.macd(stock_data['Close'])

# Drop NaN values caused by initial calculations
stock_data.dropna(inplace=True)

stock_data


C:\Users\DELL\AppData\Local\Temp\ipykernel_4504\1536852810.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stock_data['Close'].fillna(method='ffill', inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_4504\1536852810.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data['Close'].fillna(method='ffill', inplace=True)


,date,Open,High,Low,Close,Adj Close,Volume,SMA_20,EMA_50,RSI,Bollinger_High,Bollinger_Low,MACD
49,2020-03-13,66.222504,69.980003,63.237499,69.492500,67.531670,370732000,73.158374,74.813285,45.208796,83.202509,63.114240,-2.760997
50,2020-03-16,60.487499,64.769997,60.000000,60.552502,58.843914,322423600,72.124124,74.254039,36.826389,82.863448,61.384800,-3.339646
51,2020-03-17,61.877499,64.402496,59.599998,63.215000,61.431290,324056000,71.297374,73.821135,40.372320,82.106755,60.487993,-3.542553
52,2020-03-18,59.942501,62.500000,59.279999,61.667500,59.927456,300233600,70.335499,73.344522,39.002044,80.976341,59.694657,-3.784601
53,2020-03-19,61.847500,63.209999,60.652500,61.195000,59.468281,271857200,69.391499,72.868070,38.571570,79.754952,59.028046,-3.968803
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,2024-12-24,255.490005,258.209991,255.289993,258.200012,258.200012,23234700,246.293500,238.380336,75.682893,259.058756,233.528244,6.053678
1254,2024-12-26,258.190002,260.100006,257.630005,259.019989,259.019989,27237100,247.491499,239.189735,76.386796,260.312004,234.670995,6.288037
1255,2024-12-27,257.829987,258.700012,253.059998,255.589996,255.589996,42355300,248.524500,239.832882,67.575020,260.426425,236.622574,6.126376
1256,2024-12-30,252.229996,253.500000,250.750000,252.199997,252.199997,35557500,249.267999,240.317867,60.185319,260.088496,238.447502,5.659474


In [6]:
stock_data.isnull().sum()

date              0
Open              0
High              0
Low               0
Close             0
Adj Close         0
Volume            0
SMA_20            0
EMA_50            0
RSI               0
Bollinger_High    0
Bollinger_Low     0
MACD              0
dtype: int64

In [7]:
# Convert both date columns to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])
news_data['date'] = pd.to_datetime(news_data['date'])

# Merge stock prices with news sentiment
merged_data = pd.merge(stock_data, news_data, on='date', how='left')

# Fill missing sentiment with "neutral"
merged_data['sentiment'].fillna('neutral', inplace=True)

# Save merged dataset
df = merged_data.to_csv('merged_stock_news_data.csv', index=False)




C:\Users\DELL\AppData\Local\Temp\ipykernel_4504\507436691.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['sentiment'].fillna('neutral', inplace=True)


,date,Open,High,Low,Close,Adj Close,Volume,SMA_20,EMA_50,RSI,Bollinger_High,Bollinger_Low,MACD,sentiment,news_headline
0,2020-03-13,66.222504,69.980003,63.237499,69.492500,67.531670,370732000,73.158374,74.813285,45.208796,83.202509,63.114240,-2.760997,positive,The company 's order book stood at 1.5 bln eur...
1,2020-03-16,60.487499,64.769997,60.000000,60.552502,58.843914,322423600,72.124124,74.254039,36.826389,82.863448,61.384800,-3.339646,positive,The company said that paper demand increased i...
2,2020-03-17,61.877499,64.402496,59.599998,63.215000,61.431290,324056000,71.297374,73.821135,40.372320,82.106755,60.487993,-3.542553,positive,The world 's second largest stainless steel ma...
3,2020-03-18,59.942501,62.500000,59.279999,61.667500,59.927456,300233600,70.335499,73.344522,39.002044,80.976341,59.694657,-3.784601,positive,"Within the framework of the partnership , Noki..."
4,2020-03-19,61.847500,63.209999,60.652500,61.195000,59.468281,271857200,69.391499,72.868070,38.571570,79.754952,59.028046,-3.968803,positive,Seppala 's revenue increased by 0.2 % to EUR10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,2024-12-24,255.490005,258.209991,255.289993,258.200012,258.200012,23234700,246.293500,238.380336,75.682893,259.058756,233.528244,6.053678,positive,"In the Homeware area , Fiskars provides leadin..."
1205,2024-12-26,258.190002,260.100006,257.630005,259.019989,259.019989,27237100,247.491499,239.189735,76.386796,260.312004,234.670995,6.288037,neutral,"In the survey , Goodyear is the third most val..."
1206,2024-12-27,257.829987,258.700012,253.059998,255.589996,255.589996,42355300,248.524500,239.832882,67.575020,260.426425,236.622574,6.126376,neutral,"In today s business , you have to pre-empt wha..."
1207,2024-12-30,252.229996,253.500000,250.750000,252.199997,252.199997,35557500,249.267999,240.317867,60.185319,260.088496,238.447502,5.659474,neutral,Ingen is an established medical device manufac...
